# Demonstrate the Classification of new Emails using Existing Model
---
TODO:
- Use the same pre-processing/cleaning functions used before training the model and apply it to sample email.
- Use joblib to save model for future use.

## Imports

In [1]:
import joblib
import pandas as pd
from utils import clean_text
from sentence_transformers import SentenceTransformer

/Users/prokope/Projects/PsyberWatch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Model

In [2]:
model = joblib.load("./models/svm_model.joblib")

## Clean & Encode Test Email

In [3]:
email_dataframe = pd.read_feather("./data/2_clean_email_dataset.feather")

mapping = dict(zip(
    email_dataframe["Email Type"].astype("category").cat.codes.unique(), email_dataframe["Email Type"].unique()
))

In [5]:
# Retrieve device variable from 4. Analyze using SVM.ipynb
%store -r device

encoder = SentenceTransformer("all-MiniLM-L6-v2", device=device)

def encode_email(email_text):
    cleaned = clean_text(email_text)
    embedding = encoder.encode(
        cleaned,
        convert_to_numpy=True
    ).reshape(1, -1)
    return embedding

def classify_email(email_text):
    embedding = encode_email(email_text)
    prediction = model.predict(embedding)[0]
    return mapping[prediction]

## Test

In [6]:
test_email = """
URGENT: Your account has been compromised. 
Click the link below to verify your identity immediately.
"""

result = classify_email(test_email)
print("Predicted class:", result)

Predicted class: phish


In [7]:
test_email = """
Hey steve, I'm sorry to hear that you're having trouble with your account. 
I'll do my best to help you out.
"""

result = classify_email(test_email)
print("Predicted class:", result)

Predicted class: ham


In [8]:
test_email = """
Enter the code below to verify your identity:
"""

result = classify_email(test_email)
print("Predicted class:", result)

Predicted class: ham


In [30]:
test_email = """
You've been selected for a special promotion! 
Click here to claim your prize:
"""

result = classify_email(test_email)
print("Predicted class:", result)

Predicted class: phish


In [31]:
test_email = """
Are you in need of a new bike? Check out our latest collection:
"""

result = classify_email(test_email)
print("Predicted class:", result)

Predicted class: ham
